# Import ERA-5 Climate data and filter by trap locations to reduce size

In [1]:
import numpy as np
from datetime import datetime

In [2]:
import pandas as pd

In [3]:
import xarray as xr
import flox

/Users/katherinedixon/anaconda3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [4]:
state = "CA"
state_code = "CA"

In [13]:
f80 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/"+state_code+"_RH_1980-1989.nc", decode_times = True)
f90 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/"+state_code+"_RH_1990-1999.nc", decode_times = True)
#f00 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/"+state_code+"_RH_2000-2009.nc", decode_times = True)
f10 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/"+state_code+"_RH_2010-2020.nc", decode_times = True)
#f20 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/"+state_code+"_RH_2021-2023.nc", decode_times = True)

In [14]:
ca00 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/missing/"+state_code+"_RH_2000.nc", decode_times = True)
ca01 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/missing/"+state_code+"_RH_2001.nc", decode_times = True)
ca02 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/missing/"+state_code+"_RH_2002.nc", decode_times = True)
ca03 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/missing/"+state_code+"_RH_2003.nc", decode_times = True)
ca04 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/missing/"+state_code+"_RH_2004.nc", decode_times = True)
ca05 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/missing/"+state_code+"_RH_2005.nc", decode_times = True)
ca06 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/missing/"+state_code+"_RH_2006.nc", decode_times = True)
ca07 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/missing/"+state_code+"_RH_2007.nc", decode_times = True)
ca08 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/missing/"+state_code+"_RH_2008.nc", decode_times = True)
ca09 = xr.open_dataset("/Volumes/My Book/Climate/ERA_RH/missing/"+state_code+"_RH_2009.nc", decode_times = True)

In [15]:
f00 = xr.merge([ca00,ca01,ca02,ca03,ca04,ca05,ca06,ca07,ca08,ca09])

In [16]:
def downsample_df(dataset):
    max_daily = dataset.resample(time='D').max(dim='time')
    min_daily = dataset.resample(time='D').min(dim='time')
    mean_daily = dataset.resample(time='D').mean(dim='time')
    
    max_daily = max_daily.rename({'r':'max_rh'})
    min_daily = min_daily.rename({'r':'min_rh'})
    mean_daily = mean_daily.rename({'r':'mean_rh'})
    
    merged_data = xr.merge([max_daily,min_daily,mean_daily])
    
    # faster to do this on the xarray
    merged_data['year'] = merged_data['time'].dt.strftime('%Y')
    merged_data['month'] = merged_data['time'].dt.strftime('%B')
    merged_data['day'] = merged_data['time'].dt.strftime('%d')
    
    df = merged_data.to_dataframe()
    df = df.reset_index()
    
    return(df)

In [17]:
df80 = downsample_df(f80)
df90 = downsample_df(f90)
df00 = downsample_df(f00)
df10 = downsample_df(f10)
#df20 = downsample_df(f20)


In [18]:
df_all = pd.concat([df80,df90,df00,df10])

In [19]:
df_all

,time,longitude,latitude,max_rh,min_rh,mean_rh,year,month,day
0,1979-01-01,-122.0,42.00,77.785233,24.227005,59.368000,1979,January,01
1,1979-01-01,-122.0,41.75,83.191093,24.293053,60.565022,1979,January,01
2,1979-01-01,-122.0,41.50,87.948494,20.730507,59.525364,1979,January,01
3,1979-01-01,-122.0,41.25,67.810120,15.772968,45.791149,1979,January,01
4,1979-01-01,-122.0,41.00,60.426846,21.386978,42.997654,1979,January,01
...,...,...,...,...,...,...,...,...,...
3314845,2020-12-31,-116.0,35.00,28.615780,13.736099,22.055258,2020,December,31
3314846,2020-12-31,-116.0,34.75,36.376366,14.314865,24.266251,2020,December,31
3314847,2020-12-31,-116.0,34.50,36.567387,12.746780,24.855827,2020,December,31
3314848,2020-12-31,-116.0,34.25,32.453308,10.765297,23.351297,2020,December,31


In [20]:
df_all.to_csv("/Volumes/My Book/Climate/ERA_RH/downsampled/"+state+"_RH_allyears.csv")

### Filtering by traps in the state

In [5]:
trap_data = pd.read_csv("/Volumes/My Book/Synchrony/_data/ll_info_23.csv")

In [6]:
cali_data = pd.read_csv("/Volumes/My Book/Climate/ERA_RH/downsampled/California_RH_allyears.csv")
colo_data = pd.read_csv("/Volumes/My Book/Climate/ERA_RH/downsampled/Colorado_RH_allyears.csv")
pnw_data = pd.read_csv("/Volumes/My Book/Climate/ERA_RH/downsampled/PNW_RH_allyears.csv")
ari_data = pd.read_csv("/Volumes/My Book/Climate/ERA_RH/downsampled/Arizona_RH_allyears.csv")
newm_data = pd.read_csv("/Volumes/My Book/Climate/ERA_RH/downsampled/NewMexico_RH_allyears.csv")

In [7]:
df_all = pd.concat([cali_data,colo_data,pnw_data,ari_data,newm_data])

In [8]:
state_df = trap_data.reset_index(drop=True)

In [9]:
state_df

,State,manual_id,lat,lon,years_trapped
0,AZ,2,34.452258,-111.378428,15
1,AZ,15,34.429790,-111.261416,20
2,AZ,22,33.861750,-110.943743,18
3,AZ,31,33.816588,-110.926355,15
4,AZ,32,33.812968,-110.905213,2
...,...,...,...,...,...
1771,ID,2405,45.897575,-115.555061,4
1772,ID,2406,45.861841,-115.385300,4
1773,ID,2407,45.826436,-115.629222,4
1774,ID,2408,45.893359,-115.623524,4


In [11]:
trap_lats = state_df.lat
trap_lons = state_df.lon

print(trap_lats)

0       34.452258
1       34.429790
2       33.861750
3       33.816588
4       33.812968
          ...    
1771    45.897575
1772    45.861841
1773    45.826436
1774    45.893359
1775    45.617810
Name: lat, Length: 1776, dtype: float64


In [ ]:
min(trap_lats)

In [12]:
grid_lon = list(set(df_all.longitude))
grid_lat = list(set(df_all.latitude))

In [13]:
print(min(grid_lon),max(grid_lon))
print(min(grid_lat),max(grid_lat))

-123.75 -104.75
32.25 52.0


In [14]:
lat_coord = []

for t in range(0,len(trap_lats)):
    lat_coord.append(grid_lat[np.where(abs(trap_lats[t]- grid_lat) == min(abs(trap_lats[t]- grid_lat)))[0][0]])
    
lon_coord = []

for t in range(0,len(trap_lats)):
    lon_coord.append(grid_lon[np.where(abs(trap_lons[t]- grid_lon) == min(abs(trap_lons[t]- grid_lon)))[0][0]])

In [16]:
coord_df = pd.DataFrame({'lat' : trap_lats, 'lon' : trap_lons,
             'lat_coord' : lat_coord, 'lon_coord': lon_coord})

In [17]:
state_df = pd.merge(state_df,coord_df)

In [18]:
state_df["key"] = state_df["lat_coord"].astype(str) + state_df["lon_coord"].astype(str)

In [19]:
state_keys = list(set(state_df.key))

In [20]:
df_all["key"] = df_all["latitude"].astype(str) + df_all["longitude"].astype(str)

In [21]:
trap_all = df_all[df_all['key'].isin(state_keys)]

In [22]:
trap_all

,Unnamed: 0,time,longitude,latitude,max_rh,min_rh,mean_rh,year,month,day,expver,key
0,0,1979-01-01,-122.00,42.00,77.785230,24.227005,59.368000,1979,January,1,NaN,42.0-122.0
38,38,1979-01-01,-121.75,40.75,74.398820,31.522211,60.679028,1979,January,1,NaN,40.75-121.75
72,72,1979-01-01,-121.50,40.50,81.680016,25.483902,63.326480,1979,January,1,NaN,40.5-121.5
73,73,1979-01-01,-121.50,40.25,75.067300,25.874180,58.277393,1979,January,1,NaN,40.25-121.5
99,99,1979-01-01,-121.25,42.00,92.617830,38.046867,70.076070,1979,January,1,NaN,42.0-121.25
...,...,...,...,...,...,...,...,...,...,...,...,...
1963629,514285,2020-12-31,-105.75,33.00,43.205856,11.945610,32.822468,2020,December,31,NaN,33.0-105.75
1963630,514286,2020-12-31,-105.75,32.75,42.080000,13.171654,32.512000,2020,December,31,NaN,32.75-105.75
1963632,514288,2020-12-31,-105.50,36.25,103.036950,26.323318,73.642136,2020,December,31,NaN,36.25-105.5
1963645,514301,2020-12-31,-105.50,33.00,41.971897,9.765091,31.288370,2020,December,31,NaN,33.0-105.5


In [23]:
trap_all.to_csv("/Volumes/My Book/Climate/ERA_RH/downsampled/trap_rh_data_allT.csv")

### Missing small piece of Oregon

In [15]:
state_code = "OR"

In [16]:
f80 = xr.open_dataset("/Volumes/My Book/Climate/missing_updates/"+state_code+"_RH_missing1_1980-1989.nc", decode_times = True)
f90 = xr.open_dataset("/Volumes/My Book/Climate/missing_updates/"+state_code+"_RH_missing1_1990-1999.nc", decode_times = True)
f00 = xr.open_dataset("/Volumes/My Book/Climate/missing_updates/"+state_code+"_RH_missing1_2000-2009.nc", decode_times = True)
f10 = xr.open_dataset("/Volumes/My Book/Climate/missing_updates/"+state_code+"_RH_missing1_2010-2020.nc", decode_times = True)

In [19]:
df80 = downsample_df(f80)
df90 = downsample_df(f90)
df00 = downsample_df(f00)
df10 = downsample_df(f10)

In [21]:
df_all = pd.concat([df80,df90,df00,df10])

In [ ]:
df_all["key"] = df_all["latitude"].astype(str) + df_all["longitude"].astype(str)

In [ ]:
trap_all = df_all[df_all['key'].isin(state_keys)]

In [33]:
trap_all.to_csv("/Volumes/My Book/Climate/ERA_RH/downsampled/trap_rh_data_missing.csv")